In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import torch
import torch.optim as optim
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import MNIST
from torch.autograd import Variable
from torch.utils.data import Dataset, TensorDataset

import time
import math
import statistics
import random
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from sklearn.preprocessing import LabelBinarizer
from MNIST_FL import FedMLFunc as fl
from MNIST_FL import Net as Net
from tqdm import tqdm
fl = fl()

from collections import OrderedDict
from collections import defaultdict
from typing import List, Tuple

from LDP_Functions import SRR
from LDP_Functions import LDP_FL
from LDP_Functions import GRR

print("numpy", np.__version__)
print("torch", torch.__version__)
print("torchvision", torchvision.__version__)

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


numpy 1.24.4
torch 1.12.1
torchvision 0.13.1


In [ ]:
# Checking the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

# Getting GPU usage information before computation
if device.type == 'cuda':
    for i in range(0,torch.cuda.device_count()):
        print(torch.cuda.get_device_name(i))
        print('Memory Usage of device :', i)
        print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
        print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

In [ ]:
CLASSES = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
np.random.seed(25)
num_classes = 10 

# Client training settings
localepochs = 12 # The number of epochs for local model training. 50 is the default value  
BATCH_SIZE = 16  
weight_decay = 1e-4

# FL settings
num_of_clients = 500
num_selected = 500
num_rounds = 50 
epochs = 10 # The number of epochs for the clients during FL 

In [ ]:
def load_datasets():
    
    train_dataset = MNIST('./dataset', train=True, download=True, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ]))

    test_dataset = MNIST('./dataset', train=False, download=True, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ]))

    n_samples = len(train_dataset) // num_of_clients
    class_counts = torch.zeros(10)
    for i in range(len(train_dataset)):
        class_counts[train_dataset[i][1]] += 1

    # Divide the samples for each class into n parts
    class_indices = {}
    for i in range(len(train_dataset)):
        label = train_dataset[i][1]
        if label not in class_indices:
            class_indices[label] = []
        class_indices[label].append(i)

    for label in class_indices:
        np.random.shuffle(class_indices[label])
        class_indices[label] = [class_indices[label][i::num_of_clients] for i in range(num_of_clients)]

    # Create datasets for each client by combining the parts from each class
    datasets_list = []
    for i in range(num_of_clients):
        indices = []
        for label in class_indices:
            indices += class_indices[label][i]
        dataset = torch.utils.data.Subset(train_dataset, indices)
        dataloader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
        datasets_list.append(dataloader)

    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

    return datasets_list, test_loader


trainloaders, testloader = load_datasets()

In [ ]:
from torchsummary import summary

net = Net()
net.to(device)
summary(net, (1, 28, 28))

In [ ]:
torch.cuda.empty_cache()

# Instantiate models and optimizers

# Global Model
global_model = nn.DataParallel(Net()).cuda()
global_copy = nn.DataParallel(Net()).cuda()

# Client Models as a list
client_models = [nn.DataParallel(Net()).cuda() for _ in range(num_of_clients)]

# Initializing client models with global model weights and then saving them as model_x.ckpt where x stands for it's ID
for i, model in enumerate(client_models):
    model.load_state_dict(global_model.state_dict())

global_copy.load_state_dict(global_model.state_dict())

# Optimizers as a list
opt = [optim.SGD(model.parameters(), lr=0.1, momentum=0.5) for model in client_models]


if device.type == 'cuda':
    for i in range(0,torch.cuda.device_count()):
        print(torch.cuda.get_device_name(i))
        print('Memory Usage of device :', i)
        print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
        print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')


In [ ]:
from LDP_Functions import SRR
from LDP_Functions import LDP_FL
from LDP_Functions import GRR
from LDP_Functions import LDPFL


# ldp_func = SRR(c = 0.0, r = 0.075, epsilon = 5, delta_d = 20 , precision = 4, m = 10)
ldp_func = LDPFL(c = 0.0, r = 0.075, epsilon = 20)

In [ ]:
acc_train_collect = []
acc_test_collect = []
loss_train_collect = []
loss_test_collect = []

index = 0

fl.initialize_history(global_model)

for r in tqdm(range(num_rounds)):
    
    index += 1
    
    # select (num_of_clients - 1) clients randomly
    client_idx = np.random.permutation(num_of_clients)[:num_selected]
    
    trainloss = 0
    trainacc = 0
    loss = 0
    
    count = 0
    
    # clients update
    for i in client_idx:  
        count += 1
        
        
        # calling client_update
        [loss,acc]= fl.client_update(client_models[i], opt[i], trainloaders[i], epochs, device)  
        
        trainloss += loss
        trainacc += acc
        
        torch.cuda.empty_cache()
    
    # server aggregate
    
    start = time.time()
#     fl.server_aggregate_grr(global_model, [client_models[i] for i in client_idx], client_models, ldp_func)
    tot_count, method1 = fl.server_aggregate_attack(global_model, global_copy, [client_models[i] for i in client_idx], client_models, ldp_func, error_bound=0.001)
#     fl.server_aggregate_srr(global_model, [client_models[i] for i in client_idx], client_models, ldp_func)
    end = time.time()
    
    print(f"Time taken for server aggregation : {end-start} seconds.")
    print(f"-----------------------------------\nAttack Results for Round {index}\n")
    print(f"Percentage of correct estimations by method 1 : {float(method1)/tot_count}")
#     print(f"Percentage of correct estimations by method 2 (Mean of historical weights as estimate) : {float(method2)/tot_count}")
#     print(f"Percentage of estimations estimated to be right : {float(pred_count)/tot_count}\n")
    print(f"-----------------------------------\n")
    
if device.type == 'cuda':
    for i in range(0,torch.cuda.device_count()):
        print(torch.cuda.get_device_name(i))
        print('Memory Usage of device :', i)
        print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
        print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

print("Training and Evaluation completed!") 

In [ ]:
import random 
import math 
def perturb_value(c, r, epsilon): 
    # Generate a random value within the range [c-r, c+r] 
    w = random.uniform(c-r, c+r) 
    result1 = 0 
    result2 = 0 
    for i in range (1000): 
        # Compute the perturbation probability 
        prob = (w - c) * (math.exp(epsilon)-1) + r * (math.exp(epsilon)+1) 
        prob /= 2 * r * (math.exp(epsilon)+1) 
        # Generate a random number to decide whether to perturb positively or negatively 
        if random.random() <= prob: 
            perturbed_value = c + r * (math.exp(epsilon)+1) / (math.exp(epsilon)-1) 
            result1 = result1 +1 
        else: 
            perturbed_value = c - r * (math.exp(epsilon)+1) / (math.exp(epsilon)-1) 
            result2 = result2 +1 
    return w, result1/result2 

original_w = [] 
estimated_w = [] 
c=100 
r=90 
T=0 
epsilon = 1 
for i in range(100): 
    w, ratio = perturb_value(c, r, epsilon) 
    original_w.append(w) 
    w_est = (0.5*ratio-0.5)/(1+ratio)*2*r*(math.exp(epsilon)+1)/(math.exp(epsilon)-1)+c 
    estimated_w.append(w_est) count = 0 for i in range(100): if original_w[i]-estimated_w[i]<= T: count = count+1 print(count/100)